# Billboard Top 100 Scraping In Conjunction with Spotify

The motivation is to find the artists who have appeared in Billboard's Top 100 lists for the last nine years and to look for insights into the features of their current top songs. 

The first step in this process is to scrape the list of artists for each year from Billboard's website and to pass those into Spotify to retrieve their unique ids and the features of their top songs.

In [12]:
# Example of access tokens to pass the scripts, these expire periodically and must be refreshed
artist_token = 'BQDqV7wAsWS5fipMG7muoSQ8S_PYjguw_ZndC-IeHA9acFrI93oAeMgP_QONGvkvgtGEnwribF3KSBifbwmqhPRY_TKVFeQKOI8GJzafgmIs1n-RcqVncxQ_AFvzoiyAtkiC1z2ydIknjQ'
track_token ='BQATiKj1zxc6enn5CallRSLmFGFl-VziO_9eRtcSVk4HXMyP4ypM8lQshyPMn2YC2LcIxEIYdx5mz8_5Td7x-a40UtZRYfRaSA5489aTvWRn2r5_0uswTV3y_a-odN-wjm-uEFUylX2tYw'
features_token='BQAc9J_LbMT_TjMMY73atPYl4A3-P1On-frrr5hkH3kvIAx4q6U502DSu9Bcn3fwcqzg3IYTIy_NXbMlDwb4rMGYCwCV8GZnSQ0rhiEFYXw9PmFP3Ulu0oqCT6CwuJp3b-LL2AWNtvsx-YkZJJGytUDYF8g_zRARDA'

## Scraper Go Scraping

The following loop will run the billboard-scraper script for the last nine years and save the necessary data in a json file corresponding to each year for later joining.

In [ ]:
for idx in range(2010,2019):
    !eval {f"python billboard_scraper.py {idx} {artist_token} {track_token}"}

### What did the scraping look like?
The following code shows the code to locate the 100 artists' name once Billboard's webpage has been downloaded as a text file.

In [1]:
from bs4 import BeautifulSoup
import requests

artist_list = []
artist_list_clean = []
def topartists(page_url):
    url = page_url
    html = requests.get(url)
    #get html content for the page requested
    html_content = BeautifulSoup(html.content, 'html.parser')
    #find the artist div class
    artist_div = html_content.findAll('div', class_="ye-chart-item__title")
    for item in artist_div:
        #get the artist name from their link text 
        artist_list.append(item.a.text)
    for item in artist_list:
        artist_list_clean.append(item.strip())
    return artist_list_clean

In [2]:
# The top 3 artists of 2018 according to Billboard
topartists('https://www.billboard.com/charts/year-end/2018/top-artists')[0:3]

['Drake', 'Post Malone', 'Ed Sheeran']

## Putting it together
Now that we have all of the artist information for each year we can put all of this information together into a single dataframe.

In [ ]:
import pandas as pd
import getpass

# This will load all of the saved dataframes just pass in your working directory path below

path='FILL/ME/OUT'

tracks_df_2010 = pd.read_json(f'file://localhost/{path}/tracks2010.txt', orient='index')
tracks_df_2011 = pd.read_json(f'file://localhost/{path}/tracks2011.txt', orient='index')
tracks_df_2012 = pd.read_json(f'file://localhost/{path}/tracks2012.txt', orient='index')
tracks_df_2013 = pd.read_json(f'file://localhost/{path}/tracks2013.txt', orient='index')
tracks_df_2014 = pd.read_json(f'file://localhost/{path}/tracks2014.txt', orient='index')
tracks_df_2015 = pd.read_json(f'file://localhost/{path}/tracks2015.txt', orient='index')
tracks_df_2016 = pd.read_json(f'file://localhost/{path}/tracks2016.txt', orient='index')
tracks_df_2017 = pd.read_json(f'file://localhost/{path}/tracks2017.txt', orient='index')
tracks_df_2018 = pd.read_json(f'file://localhost/{path}/tracks2018.txt', orient='index')



In [ ]:
# Merging the DataFrames and using suffixes to preserve identically named columns with the year information  
tracks_df_master = tracks_df_2010.merge(tracks_df_2011, on='id', 
                                          how='outer', left_index=True, right_index=True, suffixes=('_2010', '_2011'))                                                   right_index=True, suffixes=('','_2011'))
tracks_df_master = tracks_df_master.merge(tracks_df_2012, on='id', 
                                          how='outer', left_index=True, right_index=True)
tracks_df_master = tracks_df_master.merge(tracks_df_2013, on='id', 
                                          how='outer', left_index=True, right_index=True,suffixes=('_2012','_2013'))
tracks_df_master = tracks_df_master.merge(tracks_df_2014, on='id', 
                                          how='outer', left_index=True, right_index=True)
tracks_df_master = tracks_df_master.merge(tracks_df_2015, on='id', 
                                          how='outer', left_index=True, right_index=True,suffixes=('_2014','_2015'))
tracks_df_master = tracks_df_master.merge(tracks_df_2016, on='id', 
                                          how='outer', left_index=True, right_index=True)
tracks_df_master = tracks_df_master.merge(tracks_df_2017, on='id', 
                                          how='outer', left_index=True, right_index=True,suffixes=('_2016','_2017'))
tracks_df_master = tracks_df_master.merge(tracks_df_2018, on='id', 
                                          how='outer', left_index=True, right_index=True)
tracks_df_master.rename(columns = {'tracks':'tracks_2018'}, inplace = True)

### Calling Spotify
We have a pandas dataframe with all of the unique top artists from the last nine years and a list of their top ten tracks. It's now time to call Spotify and get the audio features for these tracks. We'll need to clean up the data a little bit first . . .

In [ ]:
def get_track_features(track_ids):
    """Take list of track ids and retrieve audio features for them as a list"""
    track_features = []
    url = 'https://api.spotify.com/v1/audio-features/?ids='
    access_token = 'BQA8Khl1Ae_NUVYt9Iw7wYg1fVLIK5cZptSt1PjzcK_Pz9Ae9TbrOscs_vMPhZM8n9mO8Rtoj4Oa755nixxEfQntFssHBu5f92zqemz3u7-CjQbTLyYdg42AcIoVB5Pe4SsS16qkD9L4E0Z-JZ8-3SikXC21gWcRkw'
    url_params = '%2C'.join(track_ids)
    headers = {'Authorization': 'Bearer '+features_token}
    response=requests.get(url+url_params,headers = headers)
    track_features.append(response.json())
    return track_features 

In [ ]:
df=df.drop('id',axis=1)
df_t=df.T
# Believe it or not, this appends a column with unique values for each artist's top tracks
df['top_tracks']=df_t[[artist for artist in df_t.columns]].fillna(
    method='bfill').fillna(method='ffill').sum().apply(set).apply(list)

# Map our spotify api call to the column with track ids and add as a new column
df['top_track_features'] = df['top_tracks'].map(get_track_features)

# Find the number of appearances of each artist in the last nine years and add as a new column
df['appearances'] = df.sum(axis=1, numeric_only=True).map(int)

# Let's store this df for future reference, this file has already been provided
df.to_pickle('features.pkl')